# Teste de hipóteses para tcc

In [1]:
from scipy.stats import ttest_ind
from scipy import stats
import pandas as pd
import numpy as np
import os

In [2]:
def seleciona_colunas(df):
    df = df[(df["train_method"] == "Grid Search 3 folds") & (df["selecao_feat"] == "Tree")]
    df = df[["classe", "dataset_treino", "dataset_test", "algoritmo", "f1"]]
    return df

In [3]:
def atribui_label_1(dataset_nome):
    if "05_05_05" in dataset_nome:
        return 1
    if "05_10_00" in dataset_nome:
        return 2
    if "05_00_10" in dataset_nome:
        return 3

In [4]:
def atribui_label_2(dataset_nome):
    return dataset_nome.split("_")[0].replace("-", " ").title()

### Merge resultados

In [9]:
files = [file for file in os.listdir("./resultados") if ".csv" in file]

In [10]:
exp_1_files = [file for file in files if "experimento_1" in file]

df_exp_1 = pd.DataFrame()
for file in exp_1_files:
    df_exp_1 = pd.concat([pd.read_csv("./resultados/" + file), df_exp_1])
    
df_exp_1 = df_exp_1[df_exp_1["train_method"] == "Grid Search 3 folds"]
df_exp_1 = df_exp_1[df_exp_1["selecao_feat"] == "Tree"]
df_exp_1["treino"] = df_exp_1["dataset_treino"].apply(atribui_label_1)
df_exp_1 = df_exp_1[["algoritmo", "treino", "acuracia", "precisao", "recall", "f1"]]

In [11]:
exp_2_files = [file for file in files if "experimento_2" in file]

df_exp_2 = pd.DataFrame()
for file in exp_2_files:
    df_exp_2 = pd.concat([pd.read_csv("./resultados/" + file), df_exp_2])
    
df_exp_2 = df_exp_2[df_exp_2["train_method"] == "Grid Search 3 folds"]
df_exp_2 = df_exp_2[df_exp_2["selecao_feat"] == "Tree"]
df_exp_2["treino"] = df_exp_2["dataset_treino"].apply(atribui_label_2)
df_exp_2 = df_exp_2[["algoritmo", "treino", "acuracia", "precisao", "recall", "f1"]]

### Hipótese Experimento 1

**Idealmente um modelo será treinado com amostras de tráfego classificado como iot e não iot. Dentro da categória não iot, temos dois possiveis tipos de tráfegos, aqueles advindo de dispostivos mobiles, smarthphones e aqueles advindo de computadores de propósito geral, desktops e notebooks. Neste primeiro teste buscamos verificar se um modelo de machine-learning treinado com apenas um dos tipos de tráfego não iot, possui uma acuracia diferente de um modelo treinado com os dois tipos de trafego.**  
  
Hipótese Nula          : não há diferença de f1 quando treinamos o modelo com apenas um dos dois tipos de tráfego não iot.    
Hipótese Alternativa   : há diferença de f1 quando treinamos o modelo com apenas um dos dois tipos de tráfego não iot.  
Nível de significância : 5%

In [12]:
df_exp_1.head(5)

,algoritmo,treino,acuracia,precisao,recall,f1
0,Ada Boosting,1,0.978000,0.979570,0.976364,0.977964
1,Ada Boosting,1,0.977045,0.977436,0.976636,0.977036
2,Ada Boosting,1,0.977318,0.977796,0.976818,0.977307
3,Ada Boosting,1,0.978227,0.979841,0.976545,0.978191
4,Ada Boosting,1,0.976182,0.976355,0.976000,0.976177


In [15]:
f1_1 = df_exp_1[df_exp_1["treino"] == 1]["acuracia"].values
f1_2 = df_exp_1[df_exp_1["treino"] == 2]["acuracia"].values
f1_3 = df_exp_1[df_exp_1["treino"] == 3]["acuracia"].values
    
ttest, pval = stats.ttest_rel(f1_1, f1_2)
if pval < 0.05:
    print("iot mobile, rejeitamos hipotese nula")
else:
    print("iot mobile, falhamos em rejeitar a hipotese nula")
    
ttest, pval = stats.ttest_rel(f1_1, f1_3)
if pval < 0.05:
    print("iot normal, rejeitamos hipotese nula")
else:
    print("iot normal, falhamos em rejeitar a hipotese nula")

iot mobile, rejeitamos hipotese nula
iot normal, rejeitamos hipotese nula


### Hipótese Experimento 2

**Idealmente um modelo será treinado com amostras de tráfego classificado como iot e não iot. Dentro da categória iot, existem diferentes classes de iot. Em cada conjunto de treino terá um certa classe iot excluida dos dados de treino, esta classe estara presente totalmente nos dados de teste como iot.**  
  
Hipótese Nula          : não há diferença de f1 quando há a adição de uma nova classe de dispositivo iot na rede.    
Hipótese Alternativa   : há diferença de f1 quando há a adição de um nova classe de dispositivo iot na rede.  
Nível de significância : 5%

In [80]:
def atribui_label_2(dataset_nome):
    return dataset_nome.split("_")[0]

In [81]:
df_exp_2 = pd.read_csv("./resultados/experimento-2/tcc/resultado_experimento_2-90-14-05-2021.csv")
df_exp_2["classe"] = df_exp_2["dataset_treino"].apply(atribui_label_2)
df_exp_2 = seleciona_colunas(df_exp_2)
df_exp_2.head(5)

,classe,dataset_treino,dataset_test,algoritmo,f1
0,appliences,appliences_05_05_05-90-trains.csv,appliences_05_05_05-90-test.csv,Gaussian Naive Bayes,0.784127
1,appliences,appliences_05_05_05-90-trains.csv,appliences_05_05_05-91-test.csv,Gaussian Naive Bayes,0.781838
2,appliences,appliences_05_05_05-90-trains.csv,appliences_05_05_05-92-test.csv,Gaussian Naive Bayes,0.783119
3,appliences,appliences_05_05_05-90-trains.csv,appliences_05_05_05-93-test.csv,Gaussian Naive Bayes,0.782984
4,appliences,appliences_05_05_05-90-trains.csv,appliences_05_05_05-94-test.csv,Gaussian Naive Bayes,0.780893


In [82]:
df_exp_2_base = pd.read_csv("./resultados/experimento-2/resultado_experimento_2_base_hip-90-19-05-2021.csv")
df_exp_2_base["classe"] = df_exp_2_base["dataset_test"].apply(atribui_label_2)
df_exp_2_base = seleciona_colunas(df_exp_2_base)
df_exp_2_base.head(5)

,classe,dataset_treino,dataset_test,algoritmo,f1
0,appliences,all_iot_05_05_05-90-trains.csv,appliences_05_05_05-90-test.csv,Gaussian Naive Bayes,0.784127
1,appliences,all_iot_05_05_05-90-trains.csv,appliences_05_05_05-91-test.csv,Gaussian Naive Bayes,0.781838
2,appliences,all_iot_05_05_05-90-trains.csv,appliences_05_05_05-92-test.csv,Gaussian Naive Bayes,0.783119
3,appliences,all_iot_05_05_05-90-trains.csv,appliences_05_05_05-93-test.csv,Gaussian Naive Bayes,0.782984
4,appliences,all_iot_05_05_05-90-trains.csv,appliences_05_05_05-94-test.csv,Gaussian Naive Bayes,0.780893


In [83]:
classes = df_exp_2["classe"].unique().tolist()
classes

['appliences',
 'cameras',
 'controller-hubs',
 'energy-managment',
 'health-monitor']

In [84]:
## NAO PODE SER COMPARADO COM DF1 PELO FATO DE ELE NÃO SER TESTADO NOS MESMOS DADOS QUE ESSES MODELOS
for classe in classes:
    f1_base = df_exp_2_base[(df_exp_2_base["classe"] == classe)].sort_values(["algoritmo", "dataset_test"])["f1"].values
    f1_iot = df_exp_2[(df_exp_2["classe"] == classe)].sort_values(["algoritmo", "dataset_test"])["f1"].values
    if f1_base.shape != f1_iot.shape:
        print("Error at", classe, "arrays must have the same shape")
        break
    ttest, pval = stats.ttest_rel(f1_base, f1_iot)
    if pval < 0.05:
        print("iot", classe, "rejeitamos hipotese nula")
    else:
        print("iot", classe, "falhamos em rejeitar a hipotese nula")

iot appliences rejeitamos hipotese nula
iot cameras rejeitamos hipotese nula
iot controller-hubs rejeitamos hipotese nula
iot energy-managment rejeitamos hipotese nula
iot health-monitor rejeitamos hipotese nula


In [64]:
xclasse = "appliences"
df_exp_2_base[(df_exp_2_base["classe"] == classe) & (df_exp_2_base["algoritmo"] == "Decision Trees")].sort_values(["algoritmo", "dataset_test"])["f1"].values

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [62]:
xclasse = "appliences"
df_exp_2[df_exp_2["classe"] == xclasse].sort_values(["algoritmo", "dataset_test"])

270     1.0
630     1.0
990     1.0
1350    1.0
1710    1.0
271     1.0
631     1.0
991     1.0
1351    1.0
1711    1.0
272     1.0
632     1.0
992     1.0
1352    1.0
1712    1.0
273     1.0
633     1.0
993     1.0
1353    1.0
1713    1.0
274     1.0
634     1.0
994     1.0
1354    1.0
1714    1.0
Name: f1, dtype: float64

In [12]:
# Experimento 3